In [1]:
print("Hola Luristán")
print("Penelopa")
print("holapenes")
print("holaMasPenesParaTodos")
print("HolaJulen")

Hola Luristán
Penelopa
holapenes
holaMasPenesParaTodos
HolaJulen


In [ ]:
import pandas as pd

# Define la ruta (path) a tu archivo CSV
# Usamos '../' para "subir" un nivel desde la carpeta 'notebooks/'
path = './data/movies_dataset.csv'

# Carga el CSV en un DataFrame
try:
    df = pd.read_csv(path, sep=',')
    
    # Muestra las primeras 5 filas para verificar que se cargó correctamente
    print("¡Archivo cargado con éxito!")
    print(df.head())
    print("\n=== Data Types ===")
    print(df.dtypes)
    print("\n=== Missing Values per Column ===")
    print(df.isnull().sum())

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta: {path}")
    print("Asegúrate de que el archivo CSV esté en la carpeta 'data'.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

¡Archivo cargado con éxito!
   MovieID                         Title        Genre  ReleaseYear  \
0        1          Might toward capital       Comedy         2003   
1        2         He however experience       Comedy         1988   
2        3  Star responsibility politics       Comedy         1971   
3        4                  Exactly live       Comedy         1998   
4        5      Focus improve especially  Documentary         2021   

  ReleaseDate Country   BudgetUSD  US_BoxOfficeUSD  Global_BoxOfficeUSD  \
0  28-09-2003   China  6577427.79       6613685.82          15472035.66   
1  14-02-1988     USA  1883810.10       1930949.15           3637731.12   
2  02-11-1971     USA  2468079.29       4186694.69           7165111.24   
3  06-08-1998     USA  1447311.46       2023683.92           4373820.26   
4  17-12-2021   India   900915.86       2129629.10           3113017.38   

   Opening_Day_SalesUSD  One_Week_SalesUSD  IMDbRating  RottenTomatoesScore  \
0            1778530.

DATA CLEANING

In [5]:
#MISSING VALUES

#Ignore the tuple
df_clean = df.dropna(subset=['Global_BoxOfficeUSD', 'IMDbRating'])
print(df_clean[0:5])

#Fill in the missing value manually: In general, this approach is time 
#consuming and may not be feasible given a large data set with many 
#missing values.-->Creo que como el dataset es grande, no es buena idea.

#Use a global constant to fill in the missing values
# Rellena los NaN en columnas de texto con "Unknown"
df['Director'].fillna('Unknown', inplace=True)
df['LeadActor'].fillna('Unknown', inplace=True)

   MovieID                         Title        Genre  ReleaseYear  \
0        1          Might toward capital       Comedy         2003   
1        2         He however experience       Comedy         1988   
2        3  Star responsibility politics       Comedy         1971   
3        4                  Exactly live       Comedy         1998   
4        5      Focus improve especially  Documentary         2021   

  ReleaseDate Country   BudgetUSD  US_BoxOfficeUSD  Global_BoxOfficeUSD  \
0  28-09-2003   China  6577427.79       6613685.82          15472035.66   
1  14-02-1988     USA  1883810.10       1930949.15           3637731.12   
2  02-11-1971     USA  2468079.29       4186694.69           7165111.24   
3  06-08-1998     USA  1447311.46       2023683.92           4373820.26   
4  17-12-2021   India   900915.86       2129629.10           3113017.38   

   Opening_Day_SalesUSD  One_Week_SalesUSD  IMDbRating  RottenTomatoesScore  \
0            1778530.85         3034053.32       

C:\Users\asier.vinos\AppData\Local\Temp\ipykernel_9844\4063833176.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Director'].fillna('Unknown', inplace=True)
C:\Users\asier.vinos\AppData\Local\Temp\ipykernel_9844\4063833176.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy